In [131]:
import numpy as np
import warnings
from IPython.display import Audio, display
from scipy.io import wavfile
import matplotlib.pyplot as plt
from scipy import signal
from scipy.signal import windows
hann = signal.windows.hann

og_len = 5000 # 5 seconds
channels = 2  # Stereo audio
sr = 44100 # stream rate

audio_path = "../../resources/vocals/withers1.wav"

warnings.simplefilter("ignore", wavfile.WavFileWarning)
sr_loaded, y = wavfile.read(audio_path)

# Convert to float32 and shape to (channels, samples)
waveform = y.T.astype(np.float32) / np.max(np.abs(y))  # normalize
num_samples = waveform.shape[1]

print(f"Original:")
display(Audio(waveform, rate=sr))

Original:


In [ ]:
# === PARAMETERS ===
grain_size_ms = 250
spray_ms = 10
feedback = 0.35
mix = 1.0

sr = sr_loaded
grain_size = int(sr * grain_size_ms/1000)
launch_interval = int(grain_size * 0.25)  # 75% overlap
spray = int(sr * spray_ms/1000)
window = hann(grain_size)

# === GRAIN DELAY ===
def grain_delay_stereo(
    x,
    sr,
    grain_ms=200,
    spray_ms=20,
    delay_ms=250,
    feedback=0.35,
    mix=1.0,
    density_hz=30,                     # grains per second
    pitch_jitter_semitones=4.0,        # random ± pitch
    reverse_prob=0.2,                  # 20% of grains reversed
    stereo_spread=0.7,                 # 0=mono, 1=wide random
    lp_cutoff_hz=10000.0               # filter wet signal
):
    ch, N = x.shape

    # --- derived parameters ---
    grain_size = int(sr * grain_ms / 1000)
    window = hann(grain_size)

    spray = int(sr * spray_ms / 1000)
    base_delay = int(sr * delay_ms / 1000)

    # overlap based on density
    launch_interval = max(1, int(sr / density_hz))   # launches/sec

    # delay buffer (bigger than input to allow long delay)
    delay = np.zeros((ch, N + grain_size * 4), dtype=np.float32)

    # lowpass on wet path
    if lp_cutoff_hz is not None:
        b, a = signal.butter(1, lp_cutoff_hz/(sr*0.5), 'low')
    else:
        b, a = np.array([1], float), np.array([1], float)

    rng = np.random.default_rng()

    # === MAIN GRAIN LOOP ===
    for n in range(0, N - grain_size, launch_interval):

        # -------------------------------
        # where grain will be WRITTEN (output timeline)
        # -------------------------------
        out_start = n + base_delay
        out_end   = out_start + grain_size

        if out_start >= delay.shape[1]:
            break

        # clamp grain end to buffer end
        usable = grain_size
        if out_end > delay.shape[1]:
            usable = delay.shape[1] - out_start
            out_end = delay.shape[1]

        # -------------------------------
        # where grain READS from input timeline
        # -------------------------------
        gstart = np.clip(
            n + rng.integers(-spray, spray + 1),
            0, N - grain_size - 1
        )
        gend = gstart + grain_size

        # extract grain
        grain = x[:, gstart:gend]

        # -------------------------------
        # PITCH MODULATION (per grain)
        # -------------------------------
        if pitch_jitter_semitones != 0:
            semis = rng.uniform(-pitch_jitter_semitones, pitch_jitter_semitones)
            pitch_ratio = 2.0 ** (semis / 12.0)
        else:
            pitch_ratio = 1.0

        # resample for pitch (linear interpolation)
        idx = np.arange(usable) * pitch_ratio
        idx = np.clip(idx, 0, grain_size-2)
        idx0 = idx.astype(np.int32)
        frac = idx - idx0

        pitched = (1-frac)[None,:] * grain[:,idx0] + frac[None,:] * grain[:,idx0+1]

        # -------------------------------
        # REVERSE GRAIN (probabilistic)
        # -------------------------------
        if reverse_prob > 0 and rng.random() < reverse_prob:
            pitched = pitched[:, ::-1]

        # -------------------------------
        # STEREO PAN SCATTER
        # -------------------------------
        if stereo_spread > 0 and ch == 2:
            pan = rng.uniform(-stereo_spread, stereo_spread)
            angle = (pan + 1) * 0.25 * np.pi
            L = np.cos(angle)
            R = np.sin(angle)
            pitched[0] *= L
            pitched[1] *= R

        # -------------------------------
        # APPLY WINDOW
        # -------------------------------
        windowed = pitched[:, :usable] * window[:usable]

        # -------------------------------
        # ACCUMULATE INTO DELAY BUFFER
        # -------------------------------
        delay[:, out_start:out_end] += windowed
        delay[:, out_start:out_end] += feedback * delay[:, out_start:out_end]

    # === FILTER AND MIX ===
    wet = signal.lfilter(b, a, delay[:, :N], axis=1).astype(np.float32)
    out = (1 - mix) * x + mix * wet

    # normalize if needed
    peak = np.max(np.abs(out))
    if peak > 1:
        out /= peak

    return out.astype(np.float32)

# === PROCESS ===
processed = grain_delay_stereo(
    waveform,
    sr,
    grain_ms=60,
    spray_ms=5,
    delay_ms=15,
    feedback=0.45,
    mix=1.0,
    density_hz=60,
    pitch_jitter_semitones=0.0,
    reverse_prob=0.25,
    stereo_spread=0.2,
    lp_cutoff_hz=10000
)
print("Grains:")
display(Audio(processed, rate=sr))

Grains:
